<h1>Quantum Key Distribution (BB84) Visualisation and Basic Simulation</h1>
<hr>
<i>Written by Jamie</i><br>
<p style="margin-bottom:1.5cm;">This script is to visualise and simulate, on a basic level, how the quantum key distribution protocol <a href="https://arxiv.org/abs/2003.06557">BB84</a> and error correction protocol <a href="https://link.springer.com/chapter/10.1007/3-540-48285-7_35">cascade</a> are used to create, distribute and error correct a private encryption key.</p>

<h3>Introduction</h3>
<hr>
<p style="margin-bottom:1.5cm;">The basic premise is that two people, conventionally called Alice and Bob, want to communicate a secret securely. To do this they use the BB84 protocol to generate a private encryption key that can be used to encrypt their non-quantum secret. More information can be found in <a href="https://arxiv.org/abs/2003.06557">this research paper</a>.</p>

<h3>Import packages</h3>
<hr>
<p>Import python packages: <a href="https://pypi.org/project/random2/">random</a> and the <a href="https://pypi.org/project/tabulate/">tabulate</a> module from the library that shares its name.

In [100]:
import random
from tabulate import tabulate

<h3>Properties of Transaction</h3>
<hr> 
<p>
    Prompt the user for numerical inputs for:
    <ul>
        <li>The probability of a photon being lost in transmission.</li>
        <li>The length in bits of the random string being generated and sent by the sender "Alice".</li>
        <li>Probability of an incorrect bit being received by the receiver "Bob" due to noise in the system, also known as the error rate.</li>
    </ul>
    And run input validation on these user inputs to ensure that they are in the correct format so the program doesn't break later on. If the input validation fails, ask the user for the input again, providing the reason why the input was not sufficient.
</p>

In [15]:
## Probability of photon being lost in transmission
lose_in_tx_chance=input("Probability of losing a photon in transmission [%]: ")
valid=False
while valid == False: # Input validation
    try:
        lose_in_tx_chance=int(lose_in_tx_chance)
    except:
        lose_in_tx_chance=input("-----\nError: Value must be a number\nProbability of losing a photon in transmission [%]: ")
    else:
        if lose_in_tx_chance > 100 or lose_in_tx_chance < 0:
            lose_in_tx_chance=input("-----\nError: Value must be between 0 and 100\nProbability of losing a photon in transmission [%]: ")
        else:
            valid=True
print ("Chance of photon loss on transmission: "+str(lose_in_tx_chance)+"%") # Print value for debugging or later reference

## Number of random bits being sent
no_bits=input("Number of random bits being sent: ")
valid=False
while valid == False: # Input validation
    try:
        no_bits=int(no_bits)
    except:
        no_bits=input("-----\nError: Value must be a number\nNumber of random bits being sent: ")
    else:
        if no_bits <= 0:
            no_bits=input("-----\nError: Value must be above 0\nNumber of random bits being sent: ")
        else:
            valid=True
print ("Number of random bits being sent: "+str(no_bits)) # Print value for debugging or later reference

## Probability of an incorrect bit being received due to noise (error rate)
error_rate=input("Probability of an incorrect bit being received due to noise (error rate) [%]: ")
valid=False
while valid == False: # Input validation
    try:
        error_rate=int(error_rate)
    except:
        error_rate=input("-----\nError: Value must be a number\nProbability of an incorrect bit being received due to noise (error rate) [%]: ")
    else:
        if error_rate > 100 or error_rate < 0:
            error_rate=input("-----\nError: Value must be between 0 and 100\nProbability of an incorrect bit being received due to noise (error rate) [%]: ")
        else:
            valid=True
print ("Probability of an incorrect bit being received due to noise (error rate): "+str(error_rate)+"%") # Print value for debugging or later reference

Chance of photon loss on transmission: 5%
Number of random bits being sent: 16
Probability of an incorrect bit being received due to noise (error rate): 10%


<h3>Define functions</h3>
<hr>
<p>To reduce code sprawl and to make life easier for programming, a number of functions are defined that can be recalled by the program later.<br>
These include:
<ul>
    <li>A function to convert strings to arrays which is used to convert the randomly generated strings of 0s and 1s or Ds and Rs into arrays that are easy to preform calculations on.</li>
    <li>A function to convert arrays to strings which is helpful in debugging/user output to print out easier-to-read strings of values in arrays, without the square brackets and commas.</li>
    <li>A function to generate a random array of 1s and 0s that is as long as the integer inputted into the function (used for making Alice's random sending bits).</li>
    <li>A function to generate a random array of Ds and Rs that is as long as the integer inputted into the function (used for making Alice and Bob's encoding and decoding bases).</li>
    <li>A function that returns true or false with a probability of the integer that is inputted into it (used for making a percentage of the photons get lost in transmission and for making a percentage of the received bits contain errors).</li>
</ul>
</p>

In [97]:
def string_to_array(string): # Used for user display purposes to convert each character in a string to entries in an array
    array=[]
    for i in string: # iterate through each character in the string
        array.append(i) # append each character to the array
    return array

def array_to_string(array): # Used for user display purposes to convert each entry in an array to characters in a string
    string=""
    for i in array: # iterate through each entry in the array
        string = string + i # append each entry to the string
    return string

def generate_random_bits(length): # Used to generate a random array of 1s and 0s of a given length
    binary_string = ''.join(random.choice('01') for _ in range(length)) # generate a random string of 1s and 0s of a given length
    return string_to_array(binary_string) # Convert the string to an array and return it

def generate_random_bases(length): # Used to generate a random array of Ds and Rs of a given length
    bases_string = ''.join(random.choice('DR') for _ in range(length)) # generate a random string of Ds and Rs of a given length
    return string_to_array(bases_string) # Convert the string to an array and return it

def test_percentage(prob): # Essentially a coin flip with a given probability, has the given probability of returning true 
    # generate a random float between 0 and 1
    if random.random() < (prob/100): # if the float is less than the probability as a decimal, return true
        return True
    else: # otherwise, return false
        return False

<h3>Generate Alice & Bob's Properties</h3>
<hr>
<p>
    Alice:
    <ul>
        <li>Generate the string of random bits Alice will send to Bob.</li>
        <li>Generate the random bases Alice will use to encode the random bits.</li>
        <li>Generate the encoded photon states that are being sent to Bob</li>
    </ul>
    Bob:
    <ul>
        <li>Generate the random bases Bob will use to decode the photon states received from Alice</li>
    </ul>
</p>

In [77]:
## Alice's properties
alice_bits=generate_random_bits(no_bits) # Generate the random bits that Alice will send
sending_bases=generate_random_bases(no_bits) # Generate the random bases that Alice will use to encode her bits
# Generate the photons states that Alice will send based on her random bits and bases
sending_photons=[]
for i in range(len(sending_bases)): # iterate through each bit/base pair
    if sending_bases[i] == "D" and alice_bits[i] == "0": # if the base is D and the bit is 0, append a "/" state to the photons array
        sending_photons.append("/")
    elif sending_bases[i] == "D" and alice_bits[i] == "1": # if the base is D and the bit is 1, append a "\" state to the photons array
        sending_photons.append("\\") # The "\" character is escaped in python, so we need to use "\\" to get a single "\" character
    elif sending_bases[i] == "R" and alice_bits[i] == "0": # if the base is R and the bit is 0, append a "-" state to the photons array
        sending_photons.append("-")
    elif sending_bases[i] == "R" and alice_bits[i] == "1": # if the base is R and the bit is 1, append a "|" state to the photons array
        sending_photons.append("|")
    else:
        print("Error: D, R or 0, 1 are expected") # This should never happen, but is here just in case for debugging

## Bob's properties
receiving_bases=generate_random_bases(no_bits) # Generate the random bases that Bob will use to decode Alice's bits

<h3>Decode Received Photon States</h3>
<hr>
<p>Using Bob's random string of bases, the received photon states are decoded. The probability of photons being lost in transmission is applied, then if Bob's base is correct then the received bit is "written down" correctly and if Bob's base is incorrect then there is a 50:50 chance that the bit is "written down" incorrectly.</p>

In [79]:
received_bits=[]
for i in range(len(sending_bases)): # iterate through each received photon state & decoding base pair
    if test_percentage(lose_in_tx_chance) == True: # if the photon is lost in transmission, append a space to the received bits array
        received_bits.append(" ")
    else:
        # Incoming: /  (0)
        if sending_photons[i] == "/":
            if receiving_bases[i] == "D": # if the photon is received on the correct base, append the bit to the received bits array
                received_bits.append("0")
            elif receiving_bases[i] == "R":
                if test_percentage(50) == True: # if the photon is received on the wrong base, there is a 50% chance of it being decoded incorrectly
                    received_bits.append("0")
                else:
                    received_bits.append("1")
            else:
                print("Error (photon = /):",receiving_bases[i],"is not an expected state [D,R] of the random receiving bases") # This should never happen, but is here just in case for debugging

        # Incoming: \  (1)
        elif sending_photons[i] == "\\":
            if receiving_bases[i] == "D": # if the photon is received on the correct base, append the bit to the received bits array
                received_bits.append("1")
            elif receiving_bases[i] == "R":
                if test_percentage(50) == True: # if the photon is received on the wrong base, there is a 50% chance of it being decoded incorrectly
                    received_bits.append("0")
                else:
                    received_bits.append("1")
            else:
                print("Error (photon = \):",receiving_bases[i],"is not an expected state [D,R] of the random receiving bases") # This should never happen, but is here just in case for debugging

        # Incoming: -  (0)
        elif sending_photons[i] == "-":
            if receiving_bases[i] == "R": # if the photon is received on the correct base, append the bit to the received bits array
                received_bits.append("0")
            elif receiving_bases[i] == "D":
                if test_percentage(50) == True: # if the photon is received on the wrong base, there is a 50% chance of it being decoded incorrectly
                    received_bits.append("0")
                else:
                    received_bits.append("1")
            else:
                print("Error (photon = -):",receiving_bases[i],"is not an expected state [D,R] of the random receiving bases") # This should never happen, but is here just in case for debugging

        # Incoming: |  (1)
        elif sending_photons[i] == "|":
            if receiving_bases[i] == "R": # if the photon is received on the correct base, append the bit to the received bits array
                received_bits.append("1")
            elif receiving_bases[i] == "D":
                if test_percentage(50) == True: # if the photon is received on the wrong base, there is a 50% chance of it being decoded incorrectly
                    received_bits.append("0")
                else:
                    received_bits.append("1")
            else:
                print("Error (photon = |):",receiving_bases[i],"is not an expected state [D,R] of the random receiving bases") # This should never happen, but is here just in case for debugging

        else:
            print("Error:",sending_photons[i],"is not an expected state [-,|,/,\\] of of transmission photons") # This should never happen, but is here just in case for debugging

<h3>Public Discussion</h3>
<hr>
<p>During the public discussion stage, Bob and Alice compare bases and when they both have the same base, both can note down that the associated bit is correct. This allows them to check which bits were received correctly without releasing the bits themselves to the open public.<br>
From these correctly received bits a private key can be produced.</p>

In [99]:
public_dis=[]
for i in range(len(sending_bases)): # iterate through each encoding/decoding base pair
    if received_bits[i] != " " and sending_bases[i] == receiving_bases[i]: # if the received wasn't lost in transmission and the bases match, append "OK" to the public discussion array
        public_dis.append("OK")
    else:
        public_dis.append(" ") # otherwise, append a space to the public discussion array
if len(sending_bases) != len(public_dis):
    print("Error: the public discussion array is either too long or too short") # This should never happen, but is here just in case for debugging

# Key generation
key_bit=[] # For table display purposes (includes spaces)
key_array=[] # Array of just the key bits, no spaces
for i in range(len(public_dis)):
    if public_dis[i] == "OK":
        key_bit.append(received_bits[i])
        key_array.append(received_bits[i])
    else:
        key_bit.append(" ")
if len(public_dis) != len(key_bit): # This should never happen, but is here just in case for debugging
    print("Error: the public discussion array is either too long or too short")

# Turn key array into string
key=array_to_string(key_array)
print("Key:",key)
print("Key Length (bits):",len(key)) # Print length of key

Key: 0011010
Key Length (bits): 7


<h3>Display Each Stage in a Table</h3>
<hr>
<p>A label is inserted at the start of each array so that it is labelled in the table then each stage of BB84 (represented by the data from that stage in an array) is combined into a 2D array and then displayed as a table for easy viewing of how the protocol produced that key in a stage-by-stage manner.</p>

In [93]:
print("\033[1m"+ "\033[4m" + "Quantum Key Distribution - BB84 Protocol" + "\033[0m" + "\033[0m") # bold and underline table title
# Add label to each array for tabulation
alice_bits.insert(0, "Alice's bits")
sending_bases.insert(0, "Sending bases")
sending_photons.insert(0, "Photon state")
receiving_bases.insert(0, "Receiving bases")
received_bits.insert(0, "Bob's received bits")
public_dis.insert(0, "Public discussion")
key_bit.insert(0, "Key produced")
data = [alice_bits,sending_bases,sending_photons,receiving_bases,received_bits,public_dis,key_bit]
print(tabulate(data, tablefmt="simple_grid")) # Print table
# Remove labels from each array so they can be used for further calculations
for array in data:
    array.pop(0)

Quantum Key Distribution - BB84 Protocol
┌─────────────────────┬───┬───┬────┬────┬────┬───┬───┬───┬───┬────┬───┬────┬───┬────┬───┬────┐
│ Alice's bits        │ 0 │ 1 │ 0  │ 0  │ 1  │ 0 │ 1 │ 0 │ 1 │ 1  │ 1 │ 0  │ 1 │ 1  │ 0 │ 0  │
├─────────────────────┼───┼───┼────┼────┼────┼───┼───┼───┼───┼────┼───┼────┼───┼────┼───┼────┤
│ Sending bases       │ D │ R │ R  │ R  │ D  │ D │ R │ D │ D │ D  │ R │ D  │ R │ R  │ R │ D  │
├─────────────────────┼───┼───┼────┼────┼────┼───┼───┼───┼───┼────┼───┼────┼───┼────┼───┼────┤
│ Photon state        │ / │ | │ -  │ -  │ \  │ / │ | │ / │ \ │ \  │ | │ /  │ | │ |  │ - │ /  │
├─────────────────────┼───┼───┼────┼────┼────┼───┼───┼───┼───┼────┼───┼────┼───┼────┼───┼────┤
│ Receiving bases     │ R │ D │ R  │ R  │ D  │ R │ D │ R │ R │ D  │ D │ D  │ D │ R  │ D │ D  │
├─────────────────────┼───┼───┼────┼────┼────┼───┼───┼───┼───┼────┼───┼────┼───┼────┼───┼────┤
│ Bob's received bits │ 1 │ 1 │ 0  │ 0  │ 1  │ 1 │ 1 │ 1 │ 0 │ 1  │ 0 │ 0  │ 1 │ 1  │ 0 │ 0  │
├────────

<h3>Conclusion</h3>
<hr>
<p>This implementation is not perfect and currently in the works is the inclusion of error correction via the cascade protocol.<br>
See the file named "<i>error-correction.ipynb</i>" for the current version of this code.</p>